<a href="https://colab.research.google.com/github/DipeshDhote/ANN_project/blob/main/NexTWordPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Next Word Predictor Project**

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d "projjal1/human-conversation-training-data"

Dataset URL: https://www.kaggle.com/datasets/projjal1/human-conversation-training-data
License(s): CC0-1.0
human-conversation-training-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/human-conversation-training-data.zip

Archive:  /content/human-conversation-training-data.zip
replace human_chat.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Importing library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from collections import Counter

import nltk

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [ ]:
with open('/content/human_chat.txt','r') as f:
  content = f.readlines()
  print(content)

['Human 1: Hi!\n', 'Human 2: What is your favorite holiday?\n', 'Human 1: one where I get to meet lots of different people.\n', 'Human 2: What was the most number of people you have ever met during a holiday?\n', 'Human 1: Hard to keep a count. Maybe 25.\n', 'Human 2: Which holiday was that?\n', 'Human 1: I think it was Australia\n', 'Human 2: Do you still talk to the people you met?\n', "Human 1: Not really. The interactions are usually short-lived but it's fascinating to learn where people are coming from and what matters to them\n", 'Human 2: Yea, me too. I feel like God often puts strangers in front of you, and gives you an opportunity to connect with them in that moment in deeply meaningful ways. Do you ever feel like you know things about strangers without them telling you?\n', 'Human 1: what do you mean?\n', 'Human 2: I think it\'s like a 6th sense, often seen as "cold readings" to people, but can be remarkably accurate. I once sat next to a man in a coffee and I felt a pain in 

In [ ]:
!pip install emoji

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
import emoji

# Download stopwords if not already downloaded
nltk.download('stopwords')

def clean_text(text):

  # Convert to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    #
    text = text.replace('  ','')

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    text = emoji.replace_emoji(text,'')


    return text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
cont = []
for i in content:
  if ":" in i:  # Check if the line contains a colon
    cont.append(i.split(":")[1])
  else:
    # Handle lines without a colon (e.g., skip them or append the entire line)
    # For example, to skip lines without a colon:
    pass
    # Or to append the entire line:
    # cont.append(i)

In [ ]:
ty =   " ".join(cont)

In [ ]:
cleaned_text = clean_text(ty)

In [ ]:
cleaned_text = cleaned_text.split("\n")

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
cleaned_text

[' hi',
 'what is your favorite holiday',
 'one where i get to meet lots of different people',
 'what was the most number of people you have ever met during a holiday',
 'hard to keep a count maybe ',
 'which holiday was that',
 'i think it was australia',
 'do you still talk to the people you met',
 'not really the interactions are usually shortlived but its fascinating to learn where people are coming from and what matters to them',
 'yea me too i feel like god often puts strangers in front of you and gives you an opportunity to connect with them in that moment in deeply meaningful ways do you ever feel like you know things about strangers without them telling you',
 'what do you mean',
 'i think its like a th sense often seen as cold readings to people but can be remarkably accurate i once sat next to a man in a coffee and i felt a pain in my back i asked the stranger if he had a pain it turns out that he did in the exact spot and said he pulled a muscle while dancing at a party i h

In [ ]:
from nltk.tokenize import word_tokenize

tokenize_text = []

for i in cleaned_text:
  tokenize_text.append(word_tokenize(i))

In [ ]:
# Flatten the list of lists into a single list of words
all_words = [word for sublist in tokenize_text for word in sublist]


# build vocab
vocab = {'<unk>':0}

for token in Counter(all_words).keys():
  if token not in vocab:
    vocab[token] = len(vocab)

In [ ]:
vocab

{'<unk>': 0,
 'hi': 1,
 'what': 2,
 'is': 3,
 'your': 4,
 'favorite': 5,
 'holiday': 6,
 'one': 7,
 'where': 8,
 'i': 9,
 'get': 10,
 'to': 11,
 'meet': 12,
 'lots': 13,
 'of': 14,
 'different': 15,
 'people': 16,
 'was': 17,
 'the': 18,
 'most': 19,
 'number': 20,
 'you': 21,
 'have': 22,
 'ever': 23,
 'met': 24,
 'during': 25,
 'a': 26,
 'hard': 27,
 'keep': 28,
 'count': 29,
 'maybe': 30,
 'which': 31,
 'that': 32,
 'think': 33,
 'it': 34,
 'australia': 35,
 'do': 36,
 'still': 37,
 'talk': 38,
 'not': 39,
 'really': 40,
 'interactions': 41,
 'are': 42,
 'usually': 43,
 'shortlived': 44,
 'but': 45,
 'its': 46,
 'fascinating': 47,
 'learn': 48,
 'coming': 49,
 'from': 50,
 'and': 51,
 'matters': 52,
 'them': 53,
 'yea': 54,
 'me': 55,
 'too': 56,
 'feel': 57,
 'like': 58,
 'god': 59,
 'often': 60,
 'puts': 61,
 'strangers': 62,
 'in': 63,
 'front': 64,
 'gives': 65,
 'an': 66,
 'opportunity': 67,
 'connect': 68,
 'with': 69,
 'moment': 70,
 'deeply': 71,
 'meaningful': 72,
 'ways': 

In [ ]:
vb = []
for i in cleaned_text:
  for j in i.split():
    if j in vocab:
      vb.append(vocab[j])
    else:
      vb.append(vocab['<unk>'])


In [ ]:
cleaned_text[10]

'what do you mean'

In [ ]:
def tok(sentance,vocab):

  numerical_sentence = []

  for i in sentance:
    if i in vocab:
      numerical_sentence.append(vocab[i])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence

In [ ]:
input_numerical_sentences = []

for sentance in cleaned_text:
  input_numerical_sentences.append(tok(word_tokenize(sentance.lower()),vocab))

In [ ]:
l = [[12,32,23,3,323,3],[232,2424,42],[21]]

In [ ]:
training_sequence = []
for sentence in input_numerical_sentences:

  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [ ]:
len(training_sequence)

17735

## find maximum length of a sentance

In [ ]:
k = []
for i in training_sequence:
  k.append(len(i))

max(k)

151

### Apply Zero Padding

In [ ]:
padded_training_sequance = []

for sentance in training_sequence:
  padded_training_sequance.append([0]*(max(k) - len(sentance)) + sentance )

In [ ]:
print(padded_training_sequance[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 3]


### Converting into Tensor

In [ ]:
padded_training_sequance = torch.tensor(padded_training_sequance,dtype=torch.long)

## Define X & y

In [ ]:
X = padded_training_sequance[:,:-1]
y = padded_training_sequance[:,-1]

In [ ]:
y

tensor([  3,   4,   5,  ...,  76, 455,  34])

## Create a Custom Dataclass

In [ ]:
class CustomData(Dataset):

  def __init__(self,X,y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(X)

  def __getitem__(self, index):
    return self.X[index],self.y[index]

## Create Object of class


In [ ]:
train_data = CustomData(X,y)

In [ ]:
len(train_data)

17735

### Create Object of DataLoader Class

In [ ]:
loaded_train = DataLoader(train_data,batch_size=32,shuffle=True)

In [ ]:
for i,j in loaded_train:

  print(i,j)
  break

tensor([[   0,    0,    0,  ...,  312,   14, 1246],
        [   0,    0,    0,  ...,  658,  470,  183],
        [   0,    0,    0,  ...,  268, 2018, 1806],
        ...,
        [   0,    0,    0,  ...,   26,  827,  833],
        [   0,    0,    0,  ...,  127,  145,   87],
        [   0,    0,    0,  ..., 1925,   51,   66]]) tensor([   2, 1232, 2019,   38,  491,   42,   10, 1628,  307,   23,  221,   17,
          22,   42, 1484,   32,  264,  145,    3,   11,  173, 1192,   11,  983,
         168,   53, 2660,   36,   21,  357,   26, 1001])


## Create Architecture of LSTM Network

In [ ]:
class MyLSTM(nn.Module):

  def __init__(self,vocab):
    super().__init__()

    self.embedding = nn.Embedding(num_embeddings= vocab , embedding_dim= 100)
    self.LSTM = nn.LSTM(input_size= 100,hidden_size= 150,batch_first= True)
    self.FCNN = nn.Linear(150,vocab)

  def forward(self,x):
    embedded_out = self.embedding(x)
    intermediate_hidden_states, (final_hidden_state, final_cell_state)  = self.LSTM(embedded_out)
    out = self.FCNN(final_hidden_state.squeeze(0))

    return out






In [ ]:
model = MyLSTM(len(vocab))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model.to(device)

MyLSTM(
  (embedding): Embedding(2776, 100)
  (LSTM): LSTM(100, 150, batch_first=True)
  (FCNN): Linear(in_features=150, out_features=2776, bias=True)
)

### Other parameters

In [ ]:
epochs = 50
learning_rate = 0.001

# Loss
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training Loop

In [ ]:
for epoch in range(epochs):

  total_loss = 0

  for batch_x,batch_y in loaded_train:

    batch_x,batch_y  = batch_x.to(device),batch_y.to(device)

    # zero gradient
    optimizer.zero_grad()

    # forward Pass
    out = model.forward(batch_x)

    # calculate loss
    loss = criterion(out,batch_y)

    # Backward Pass
    loss.backward()

    # calaculate gradient using optimizer
    optimizer.step()

    # loss
    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")




Epoch: 1, Loss: 3539.8772
Epoch: 2, Loss: 3089.5657
Epoch: 3, Loss: 2776.6942
Epoch: 4, Loss: 2489.4330
Epoch: 5, Loss: 2217.0804
Epoch: 6, Loss: 1964.5039
Epoch: 7, Loss: 1729.3019
Epoch: 8, Loss: 1514.2357
Epoch: 9, Loss: 1324.5510
Epoch: 10, Loss: 1158.9199
Epoch: 11, Loss: 1010.6858
Epoch: 12, Loss: 884.8018
Epoch: 13, Loss: 771.8277
Epoch: 14, Loss: 675.1559
Epoch: 15, Loss: 591.0579
Epoch: 16, Loss: 518.3466
Epoch: 17, Loss: 458.7008
Epoch: 18, Loss: 404.1991
Epoch: 19, Loss: 360.6722
Epoch: 20, Loss: 323.3933
Epoch: 21, Loss: 294.0817
Epoch: 22, Loss: 267.5260
Epoch: 23, Loss: 247.4903
Epoch: 24, Loss: 232.0386
Epoch: 25, Loss: 218.7045
Epoch: 26, Loss: 206.8147
Epoch: 27, Loss: 198.8713
Epoch: 28, Loss: 191.8059
Epoch: 29, Loss: 186.0776
Epoch: 30, Loss: 181.6177
Epoch: 31, Loss: 177.4353
Epoch: 32, Loss: 173.1526
Epoch: 33, Loss: 172.2884
Epoch: 34, Loss: 168.2358
Epoch: 35, Loss: 164.0782
Epoch: 36, Loss: 162.6981
Epoch: 37, Loss: 162.6597
Epoch: 38, Loss: 163.6397
Epoch: 39,

In [ ]:
def prediction(model,vocab,text):

  # split into words (tokenization)
  word_tokenize_text = word_tokenize(text)
  print(word_tokenize_text)

  # numerical tokenizatio
  numerical_tokinize_text = tok(word_tokenize_text,vocab)
  print(numerical_tokinize_text)

  # Padding
  padded_text = [0] * (151 - len(text)) + numerical_tokinize_text
  print(padded_text)

  # converting padded text into tensor
  tens_padded_text = torch.tensor(padded_text,dtype=torch.long).unsqueeze(0)

  print(tens_padded_text)
  print(tens_padded_text.shape)

  # Move model & input text into GPU
  model.to(device)
  tens_padded_text = tens_padded_text.to(device)


  # fit to the model
  output = model(tens_padded_text)

  value,index = torch.max(output,dim=1)

  # index value
  print(index)

  # get true text value
  # merge with text

  return text + " " + list(vocab.keys())[index]

In [ ]:
text = "do you still talk to the"

prediction(model,vocab,text)

['do', 'you', 'still', 'talk', 'to', 'the']
[36, 21, 37, 38, 11, 18]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 36, 21, 37, 38, 11, 18]
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0

'do you still talk to the people'

In [ ]:
torch.save(model.state_dict(),"model_weights_path")

In [ ]:
model.load_state_dict(torch.load("/content/model_weights_path"))
model.eval()

<ipython-input-111-4e83c279d614>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/model_weights_path"))


MyLSTM(
  (embedding): Embedding(2776, 100)
  (LSTM): LSTM(100, 150, batch_first=True)
  (FCNN): Linear(in_features=150, out_features=2776, bias=True)
)